# Win/weapon: WM 2023-03-20

## Parameters

In [30]:
from s2_analytics.constants import WEAPON_MODS_CATALOG
ACTIVE_WM = WEAPON_MODS_CATALOG.latest()

CHART_MAX_WIDTH = 10
CHART_HEIGHT_PER_ENTRY = 0.3

MINIMUM_SAMPLES = 20
MAX_IMBALANCE = 0.20
START_DATE = ACTIVE_WM.datetime
PLAYLISTS = ["CTF-Standard-6"]

print(f"Start date: {START_DATE}")
print(f"Playlists: {','.join(PLAYLISTS)}")
print()
print(f"Min samples: {MINIMUM_SAMPLES}")
print(f"Max match imbalance: {MAX_IMBALANCE * 100:.0f}% \n"
      f"    (eg worst balance: Red team win probability {(50 - MAX_IMBALANCE * 100 / 2):.1f}% and {(50 + MAX_IMBALANCE * 100 / 2):.1f}% for Blue)")


Start date: 2023-03-20 00:00:00
Playlists: CTF-Standard-6

Min samples: 20
Max match imbalance: 20% 
    (eg worst balance: Red team win probability 40.0% and 60.0% for Blue)


In [31]:
from s2_analytics.collect.summary_collector import SummaryCollector
from s2_analytics.collect.team_round_tag_collector import TeamRoundTagCorrelationAnalyzer
from pandas import DataFrame

from s2_analytics.filters import max_imbalance
import sqlite3

import pandas as pd
import seaborn as sns

from s2_analytics.collect.sqlite_collector import SqliteCollector
from s2_analytics.constants import WEAPONS_PRIMARY
from s2_analytics.importer import import_games
from s2_analytics.analyze.main_weapon_analyzer import MainWeaponRoundTagger

conn = sqlite3.connect("file::memory:")
sqlite_collector = SqliteCollector(sqlite_conn=conn).init()
summary_collector = SummaryCollector(conn, sqlite_collector)
tag_correlation_analyzer = TeamRoundTagCorrelationAnalyzer(conn, sqlite_collector, [
    MainWeaponRoundTagger([WEAPONS_PRIMARY])]).init()
import_games(
    "logs_ranked/",
    start_date=START_DATE,
    processors=[tag_correlation_analyzer, sqlite_collector],
    game_filters=[
        lambda g: g.playlist_code in ["CTF-Standard-6"],
        max_imbalance(MAX_IMBALANCE)
    ]
)
pass

## Data summary

In [32]:
import tabulate

summary = summary_collector.get_summary().to_table()
tabulate.tabulate(summary, tablefmt='html')

First game,2023-03-20
Last game,2023-03-20
Games total,5
Rounds total,12
Games in playlist `CTF-Standard-6`,5


## Aggregated result

In [33]:
from matplotlib import pyplot as plt

correlation_per_map = tag_correlation_analyzer.calculate_win_correlation()

correlation_df = pd.DataFrame(correlation_per_map.items(), columns=["tag", "corr"])
counts_df = pd.DataFrame(
    tag_correlation_analyzer.tag_counts().items(),
    columns=["tag", "cnt"])

df = pd.merge(correlation_df, counts_df, on=["tag"])


def filter_and_sort_for_corr_charts(df: DataFrame, min_samples):
    df["corr_abs"] = df['corr'].abs()
    df = df[df.apply(lambda x: x["cnt"] > min_samples, axis=1)]
    return df.sort_values("corr_abs", ascending=False) \
        .sort_values("corr", ascending=False)


df = filter_and_sort_for_corr_charts(df, MINIMUM_SAMPLES)
if df.size == 0:
    print("not enough samples :(")
else :
    fig, axes = plt.subplots(1, 2, figsize=(CHART_MAX_WIDTH, len(df["corr"]) * CHART_HEIGHT_PER_ENTRY), sharey=True)
    fig.suptitle(f"Weapon-Victory correlation - {ACTIVE_WM}", fontsize=14)
    sns.barplot(x=df["corr"], y=df["tag"], ax=axes[0]) \
        .set(xlabel="Round victory correlation coefficient", ylabel=None)
    sns.barplot(x=df["cnt"], y=df["tag"], ax=axes[1]) \
        .set(xlabel="Count of entries", ylabel=None)

pass

not enough samples :(


## WeaponMod changes

In [34]:
print(ACTIVE_WM.changes)